In [60]:
import os
import json
import dotenv
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv(dotenv_path=".env", verbose=True, override=True)


client = AzureOpenAI(
    api_key="eadf76dd169e4172a463e7375946835f",  
    api_version="2024-02-15-preview",
    azure_endpoint = "https://green-code-uks.openai.azure.com"
    )


### Create assistant without Reference file

In [61]:
# Create an assistant
assistant = client.beta.assistants.create(
  name = 'Green IT Code Writer 2',
  instructions=f"You are a helpful AI assistant who re-factors the code from an uploaded file to make it more efficient" 
    f"You have access to a sandboxed environment for writing and testing code."
    f"You should follow these steps:"
    f"1. Re-write the code in the same language as the origninal code."
    f"2. Test the re-written code and ensure it functions correctly and same as the original code."
    f"3. Run the code to confirm that it runs successfully"
    f"4. If the code runs successfully, share the code as a file that can be downloaded"
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  model="code",
  tools=[{"type": "code_interpreter"}]
)

In [62]:
#Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_yw62FSD9M55sY0KNFm1eogEk', created_at=1718361934, metadata={}, object='thread', tool_resources=None)


In [63]:
import os

# Get the current file path from the environment variable
current_file = os.environ['CURRENT_FILE']

# Upload the current file
with open(current_file, "rb") as f:
    uploaded_file = client.files.create(
        file=f,
        purpose='assistants'
    )
uploaded_files = [uploaded_file]

In [64]:
import time

prompts = ["Make the code energy efficient", "Provide a version of this function that is optimized for energy efficiency ", "Optimize this code to use less CPU resources"]  # Add your prompts here
for uploaded_file in uploaded_files:
    for prompt in prompts:
        thread = client.beta.threads.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                    "file_ids": [uploaded_file.id]
                }
            ]
        )

        # Wait for the thread to finish and get the assistant's responses
        while True:
            thread = client.beta.threads.get(thread.id)
            if thread.is_done:
                break
            time.sleep(1)  # Wait for a short time before checking again

        # Print the assistant's responses
        for message in thread.messages:
            if message.role == 'assistant':
                print(message.content)

In [65]:
#Check messages in the thread
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_fpJj9pPEPHBSWzLN3Z5QOEtD",
      "assistant_id": null,
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Optimize this code to use less CPU resources"
          },
          "type": "text"
        }
      ],
      "created_at": 1718362039,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_36DZtdxe3IUU3Vyqwc84GHU6",
      "file_ids": [
        "assistant-050kEcrUE4cVSP6ytqbqm13P"
      ]
    }
  ],
  "object": "list",
  "first_id": "msg_fpJj9pPEPHBSWzLN3Z5QOEtD",
  "last_id": "msg_fpJj9pPEPHBSWzLN3Z5QOEtD",
  "has_more": false
}


In [68]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

BadRequestError: Error code: 400 - {'error': {'message': 'Thread thread_36DZtdxe3IUU3Vyqwc84GHU6 already has an active run run_Pka4lnP7R5QXJCD4BUdckHHT.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [75]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


In [76]:
print(run.last_error)
print(file)

None
update_google_sheets.py


In [77]:
#Print messages in the thread post run
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_1tpaJXJbuaReLKzhZtkAQDT9",
      "assistant_id": "asst_hqfXrQaWLEevLfTrtIaDWOHa",
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [
              {
                "end_index": 158,
                "file_path": {
                  "file_id": "assistant-Q1BaKw2xAB5RAsgf0RYj4XwA"
                },
                "start_index": 122,
                "text": "sandbox:/mnt/data/refactored_code.py",
                "type": "file_path"
              }
            ],
            "value": "The refactored code has been saved to a file. You can download the file using the link below:\n\n[Download Refactored Code](sandbox:/mnt/data/refactored_code.py)\n\nPlease make sure to set up the necessary environment variables (`GOOGLE_APPLICATION_CREDENTIALS`, `CSV_FILE_PATH`, and `EXCEL_FILE_PATH`) before running the refactored code in your environment with `gspread` installed. This will

In [78]:
#Extract the content of the latest question only
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['annotations'][0]['file_path']['file_id']
print(code)

assistant-Q1BaKw2xAB5RAsgf0RYj4XwA


In [80]:
# Download the refined file
download_directory = "D:\\Documents\\TechM\\Green_Software_Development\\Third Task\\Projects & Docs\\Assistant api\\Refined Files"
content = client.files.content(uploaded_file)

original_filename = os.path.basename(current_file)  # Extract the filename from the original file path
new_filename = "refined_" + original_filename  # Prepend "refined_" to the filename
code_file = content.write_to_file(os.path.join(download_directory, new_filename))

### Clean up

In [ ]:
client.beta.assistants.delete(assistant.id)
client.beta.threads.delete(thread.id)